In [2]:
# !pip install yfinance
import yfinance as yf

In [3]:
# !pip install transformers
import transformers

/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [2]:
accuracy_map = {}

In [4]:
start = datetime.datetime(2008,8,8)
end = datetime.datetime(2014,7,1)
test_start = datetime.datetime(2014,7,2)
test_end = datetime.datetime(2016,7,1)

def prepare_data(data, days=30):
    X, y, date = [], [], []
    for i in range(len(data) - days):
        X.append(data['Close'].values[i:i+days])
        y.append(data['Close'].values[i+days])
        date.append(np.datetime_as_string(data['Date'].values[i+days], unit='D'))
    return np.array(X), np.array(y),date

def accuracy(truth,predictions):
    correct = 0
    total = 0
    for i in range(1,len(truth)):
        total += 1
        if (truth[i]-truth[i-1])>0 and (predictions[i]-truth[i-1])>0 or  (truth[i]-truth[i-1])<=0 and (predictions[i]-truth[i-1])<=0 :
            correct += 1
        else:
            continue
            # print(truth[i-1],truth[i],predictions[i])
    # print(f"accuracy{correct/total}, mse{np.mean((truth - predictions) ** 2)}")
    
    return correct/total,np.mean((truth - predictions) ** 2)

def predict_LR(stock_name,company_name):
    train = yf.download(stock_name, start, end)
    test = yf.download(stock_name, test_start, test_end)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    train["Company"]=company_name
    test["Company"]=company_name

    # Prepare the data
    X_train, y_train, date_train = prepare_data(train)
    X_test, y_test,date_test = prepare_data(test)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # stock_price_map = {key: value for key, value in zip(date_test, y_train)}
    stock_prediction_map = {key: value for key, value in zip(date_test, predictions)}
    

    # Evaluate the model
    # plt.figure(figsize=(10, 5))
    # plt.plot(y_test, label='True Prices')
    # plt.plot(predictions, label='Predicted Prices')
    # plt.legend()
    # plt.title('Stock Price Prediction')
    # plt.show()


    accuracy_map[company_name] = [accuracy(y_test,predictions)]
    return stock_prediction_map
        

In [6]:
company_map = {'Amazon': 'AMZN', 'Meta': 'META', 'Google': 'GOOGL', 'Microsoft': 'MSFT', 'Apple': 'AAPL', 'IBM': 'IBM', 'NVIDIA': 'NVDA', 'Tesla': 'TSLA', 'Salesforce': 'CRM', 'AMD': 'AMD'}
# predict_LR("META","META")
# Switch keys and values
stock_prediction_map = {}
for key, value in company_map.items():
    stock_prediction_map[key] = predict_LR(value,key)
# stock_prediction_map

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2734537752.py:19: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1]) == (predictions[i]-truth[i-1])/abs(predictions[i]-truth[i-1]):
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2734537752.py:19: RuntimeWarning: invalid value encountered in scalar divide
  if (truth[i]-truth[i-1])/abs(truth[i]-truth[i-1

In [7]:
accuracy_map

{'Amazon': [(0.5221987315010571, 0.27138410217779574)],
 'Meta': [(0.46511627906976744, 2.9811482053333807)],
 'Google': [(0.5031712473572939, 0.2712143662161038)],
 'Microsoft': [(0.5052854122621564, 0.6500360714208605)],
 'Apple': [(0.4608879492600423, 0.20987337371763637)],
 'IBM': [(0.49471458773784355, 4.022022075505506)],
 'NVIDIA': [(0.46511627906976744, 0.00024436089268023816)],
 'Tesla': [(0.4693446088794926, 0.16787077068440348)],
 'Salesforce': [(0.5243128964059197, 2.003914772152267)],
 'AMD': [(0.4630021141649049, 0.014939979324329859)]}

semantic analysis example

In [8]:
from transformers import BertTokenizer, BertModel, pipeline

import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example sentence
sentence = "The quick brown fox jumps over the lazy dog."

# Tokenize the input sentence and get token IDs
inputs = tokenizer(sentence, return_tensors='pt')

# Get the outputs from the model
with torch.no_grad():
    outputs = model(**inputs)

# The last hidden state of the [CLS] token (sentence-level representation)
sentence_embedding = outputs.last_hidden_state[:, 0, :]
# sentiment_analysis = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')
sentiment_analysis = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)
sentences = [ "I love this product! It's amazing.","Google"]
score_map = {"positive":1,"neutral":0, "negative":-1}
# Analyze the sentiment of each sentence
for sentence in sentences:
    result = sentiment_analysis(sentence)[0]
    print(result)
    print(f"Sentence: {sentence}")


/Users/yanran/miniconda3/envs/hw2/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'positive', 'score': 0.9930960536003113}, {'label': 'neutral', 'score': 0.005177223589271307}, {'label': 'negative', 'score': 0.0017267182702198625}]
Sentence: I love this product! It's amazing.
[{'label': 'positive', 'score': 0.5264188647270203}, {'label': 'neutral', 'score': 0.23904329538345337}, {'label': 'negative', 'score': 0.23453785479068756}]
Sentence: Google


cosine similarity example

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')

def compute_embeddings(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # Compute token embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Mean pooling
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example texts
text1 = "Microsoft"
text2 = "Musharraf to be impeached."

# Compute embeddings for both texts
embedding1 = compute_embeddings(text1)
embedding2 = compute_embeddings(text2)

# Convert embeddings to numpy arrays
embedding1_np = embedding1.numpy()
embedding2_np = embedding2.numpy()

# Compute cosine similarity
similarity = cosine_similarity(embedding1_np, embedding2_np)
print(similarity)

[[0.00980911]]


In [10]:
def sentiment(title_lst,stock_name):
    res = []
    for title in title_lst: 
        if cosine_similarity(compute_embeddings(title.lower()), compute_embeddings(stock_name.lower())) > 0.2 or stock_name.lower() in title.lower():
            # result = sentiment_analysis(title)[0]
            res.append(sentiment_analysis(title))
        else:
            res.append([])
    return res


In [11]:
#read news data into news_dict
import pandas as pd
csv_file_path = 'Combined_News_DJIA.csv'
df = pd.read_csv(csv_file_path)
df = df.drop(columns=['Label'])
news_dict = df.set_index('Date').T.to_dict('list')

In [12]:
#save customized data for training
columns = ['date', 'company', 'stock price', 'sentiment result', 'prediction']
customized_data = pd.DataFrame(columns=columns)

In [13]:
start = datetime.datetime(2008,8,8)
end = datetime.datetime(2014,7,1)
test_start = datetime.datetime(2014,8,14)
test_end = datetime.datetime(2016,7,1)
# Amazon
Amazon = yf.download("AMZN", test_start, test_end)

# Meta
Meta = yf.download("META", test_start, test_end)

# Google
Google = yf.download("GOOG", test_start, test_end)

# Microsoft
Microsoft = yf.download("MSFT", test_start, test_end)

# Apple
Apple = yf.download("AAPL", test_start, test_end)

# IBM
IBM = yf.download("IBM", test_start, test_end)

# Meta
NVIDIA = yf.download("NVDA", test_start, test_end)

# Tesla
Tesla = yf.download("TSLA", test_start, test_end)

# Salesforce
Salesforce = yf.download("CRM", test_start, test_end)

# AMD
AMD = yf.download("AMD", test_start, test_end)

Amazon.reset_index(inplace=True)
Meta.reset_index(inplace=True)
Google.reset_index(inplace=True)
Microsoft.reset_index(inplace=True)
Apple.reset_index(inplace=True)
IBM.reset_index(inplace=True)
NVIDIA.reset_index(inplace=True)
Tesla.reset_index(inplace=True)
Salesforce.reset_index(inplace=True)
AMD.reset_index(inplace=True)

Amazon["Company"]='Amazon'
Meta["Company"]='Meta'
Microsoft["Company"]='Microsoft'
Apple["Company"]='Apple'
Google["Company"]='Google'
IBM["Company"]='IBM'
NVIDIA["Company"]='NVIDIA'
Tesla["Company"]='Tesla'
Salesforce["Company"]='Salesforce'
AMD["Company"]='AMD'

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [14]:
company_map = {
    # "Amazon": Amazon,
    # "Meta": Meta,
    "Google": Google,
    "Microsoft": Microsoft,
    "Apple": Apple,
    "IBM": IBM,
    "NVIDIA": NVIDIA,
    "Tesla": Tesla,
    "Salesforce": Salesforce,
    "AMD": AMD
}
# Display the dictionary
columns = ['date', 'company', 'stock price', 'sentiment result', 'prediction']
for k,v in company_map.items():
    #initialize df
    customized_data = pd.DataFrame(columns=columns)

    company_name = k
    dataframe = v
    for index, row in dataframe.iterrows():
        print(row)
        print(f"Date: {row['Date']}")
        print(f"Company: {row['Company']}")
        new_row = {
        'date': row['Date'],
        'company': company_name,
        'stock price': row['Close'],
        'sentiment result': sentiment(news_dict[row['Date'].strftime('%Y-%m-%d')],company_name),
        'prediction':stock_prediction_map[company_name][row['Date'].strftime('%Y-%m-%d')]
        }
        print(sentiment(news_dict[row['Date'].strftime('%Y-%m-%d')],company_name))
        customized_data = customized_data._append(new_row, ignore_index=True)
    customized_data.to_csv(f'{company_name}.csv', index=False)  
customized_data

Date         2014-08-14 00:00:00
Open                   28.730122
High                   28.815886
Low                    28.465847
Close                  28.653831
Adj Close              28.653831
Volume                  19709966
Company                   Google
Name: 0, dtype: object
Date: 2014-08-14 00:00:00
Company: Google
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-15 00:00:00
Open                    28.81389
High                   28.889683
Low                    28.447895
Close                  28.595491
Adj Close              28.595491
Volume                  30383189
Company                   Google
Name: 1, dtype: object
Date: 2014-08-15 00:00:00
Company: Google


/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2167295310.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  customized_data = customized_data._append(new_row, ignore_index=True)


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-18 00:00:00
Open                   28.726631
High                   29.145481
Low                    28.721146
Close                  29.028303
Adj Close              29.028303
Volume                  25682318
Company                   Google
Name: 2, dtype: object
Date: 2014-08-18 00:00:00
Company: Google
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [[{'label': 'positive', 'score': 0.5698007941246033}, {'label': 'neutral', 'score': 0.1665957272052765}, {'label': 'negative', 'score': 0.26360347867012024}]], [], [[{'label': 'positive', 'score': 0.7075182795524597}, {'label': 'neutral', 'score': 0.11710415035486221}, {'label': 'negative', 'score': 0.1753775179386139}]], [], [], [], [], []]
Date         2014-08-19 00:00:00
Open                   29.169914
High                   29.286594
Low                     29.12005
Close                  29

/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2167295310.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  customized_data = customized_data._append(new_row, ignore_index=True)


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-18 00:00:00
Open                   44.939999
High                   45.110001
Low                        44.68
Close                  45.110001
Adj Close              38.399582
Volume                  26891100
Company                Microsoft
Name: 2, dtype: object
Date: 2014-08-18 00:00:00
Company: Microsoft
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [[{'label': 'positive', 'score': 0.7075182795524597}, {'label': 'neutral', 'score': 0.11710415035486221}, {'label': 'negative', 'score': 0.1753775179386139}]], [], [], [], [], []]
Date         2014-08-19 00:00:00
Open                   44.970001
High                       45.34
Low                    44.830002
Close                  45.330002
Adj Close              38.827873
Volume                  28139500
Company                Microsoft
Name: 3, dtype: object
Date: 2014-08-19 00:00:

...

/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2167295310.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  customized_data = customized_data._append(new_row, ignore_index=True)


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [[{'label': 'positive', 'score': 0.33308449387550354}, {'label': 'neutral', 'score': 0.23069685697555542}, {'label': 'negative', 'score': 0.43621861934661865}]], [], [], [], []]
Date         2014-08-18 00:00:00
Open                   54.110001
High                   54.630001
Low                    53.900002
Close                  54.380001
Adj Close              54.309059
Volume                   4757300
Company               Salesforce
Name: 2, dtype: object
Date: 2014-08-18 00:00:00
Company: Salesforce
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [[{'label': 'positive', 'score': 0.7075182795524597}, {'label': 'neutral', 'score': 0.11710415035486221}, {'label': 'negative', 'score': 0.1753775179386139}]], [], [], [], [], []]
Date         2014-08-19 00:00:00
Open                   54.529999
High                   55.209999
Low                        54.43
Close              

/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_54302/2167295310.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  customized_data = customized_data._append(new_row, ignore_index=True)


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-18 00:00:00
Open                        4.14
High                        4.22
Low                         4.14
Close                       4.21
Adj Close                   4.21
Volume                  45381300
Company                      AMD
Name: 2, dtype: object
Date: 2014-08-18 00:00:00
Company: AMD
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-19 00:00:00
Open                         4.2
High                         4.3
Low                          4.2
Close                       4.27
Adj Close                   4.27
Volume                  23222100
Company                      AMD
Name: 3, dtype: object
Date: 2014-08-19 00:00:00
Company: AMD
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Date         2014-08-20 00:00:00
Open        

,date,company,stock price,sentiment result,prediction
0,2014-08-14,AMD,4.08,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.118598
1,2014-08-15,AMD,4.13,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.042908
2,2014-08-18,AMD,4.21,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.127299
3,2014-08-19,AMD,4.27,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.231464
4,2014-08-20,AMD,4.28,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.298276
...,...,...,...,...,...
469,2016-06-24,AMD,4.88,"[[], [], [], [], [], [], [], [], [], [], [], [...",5.188803
470,2016-06-27,AMD,4.72,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.874105
471,2016-06-28,AMD,5.12,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.739716
472,2016-06-29,AMD,5.13,"[[], [], [], [], [], [], [], [], [], [], [], [...",5.066164


In [15]:

customized_data.to_csv('amazon.csv', index=False)  


In [16]:
customized_data

,date,company,stock price,sentiment result,prediction
0,2014-08-14,AMD,4.08,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.118598
1,2014-08-15,AMD,4.13,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.042908
2,2014-08-18,AMD,4.21,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.127299
3,2014-08-19,AMD,4.27,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.231464
4,2014-08-20,AMD,4.28,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.298276
...,...,...,...,...,...
469,2016-06-24,AMD,4.88,"[[], [], [], [], [], [], [], [], [], [], [], [...",5.188803
470,2016-06-27,AMD,4.72,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.874105
471,2016-06-28,AMD,5.12,"[[], [], [], [], [], [], [], [], [], [], [], [...",4.739716
472,2016-06-29,AMD,5.13,"[[], [], [], [], [], [], [], [], [], [], [], [...",5.066164


Hidden Markov Model

In [18]:
import pandas as pd
import numpy as np
# import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [19]:
def prepare_data(data, days=30):
    X, y, date = [], [], []
    for i in range(len(data) - days):
        X.append(data['Close'].values[i:i+days])
        y.append(data['Close'].values[i+days])
        date.append(np.datetime_as_string(data['Date'].values[i+days], unit='D'))
    return np.array(X), np.array(y),date

def accuracy(truth,predictions):
    correct = 0
    total = 0
    for i in range(1,len(truth)):
        total += 1
        if (truth[i]-truth[i-1])>0 and (predictions[i]-truth[i-1])>0 or  (truth[i]-truth[i-1])<=0 and (predictions[i]-truth[i-1])<=0 :
            correct += 1
        else:
            continue
            # print(truth[i-1],truth[i],predictions[i])
    # print(f"accuracy{correct/total}, mse{np.mean((truth - predictions) ** 2)}")
    
    return correct/total,np.mean((truth - predictions) ** 2)

def predict_LR(stock_name,company_name):
    train = yf.download(stock_name, start, end)
    test = yf.download(stock_name, test_start, test_end)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    train["Company"]=company_name
    test["Company"]=company_name

    # Prepare the data
    X_train, y_train, date_train = prepare_data(train)
    X_test, y_test,date_test = prepare_data(test)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # stock_price_map = {key: value for key, value in zip(date_test, y_train)}
    stock_prediction_map = {key: value for key, value in zip(date_test, predictions)}
    

    # Evaluate the model
    # plt.figure(figsize=(10, 5))
    # plt.plot(y_test, label='True Prices')
    # plt.plot(predictions, label='Predicted Prices')
    # plt.legend()
    # plt.title('Stock Price Prediction')
    # plt.show()


    accuracy_map[company_name] = [accuracy(y_test,predictions)]
    return stock_prediction_map
        

In [20]:
company_name = "Google"
company_df = pd.read_csv(f'{company_name}.csv')
company_df

,date,company,stock price,sentiment result,prediction
0,2014-08-14,Google,28.653831,"[[], [], [], [], [], [], [], [], [], [], [], [...",29.236557
1,2014-08-15,Google,28.595491,"[[], [], [], [], [], [], [], [], [], [], [], [...",29.247295
2,2014-08-18,Google,29.028303,"[[], [], [], [], [], [], [], [], [], [], [], [...",29.186187
3,2014-08-19,Google,29.262659,"[[], [], [], [], [], [], [], [], [], [], [], [...",29.591514
4,2014-08-20,Google,29.144484,"[[], [], [], [], [], [], [], [[{'label': 'posi...",29.965580
...,...,...,...,...,...
469,2016-06-24,Google,33.761002,"[[], [], [], [], [], [], [], [], [], [], [], [...",35.730130
470,2016-06-27,Google,33.412998,"[[], [], [], [], [], [], [], [], [], [], [], [...",34.269621
471,2016-06-28,Google,34.001999,"[[], [[{'label': 'positive', 'score': 0.265505...",34.093348
472,2016-06-29,Google,34.205502,"[[], [], [], [], [], [], [], [], [], [[{'label...",34.504956


In [13]:
def average(data):
    if len(data) <1:
        return [0,0,0]
    data = np.array(data)
    num_rows = data.shape[0]
    # Calculate weights inversely proportional to the index (higher weight for lower index)
    weights = np.arange(num_rows, 0, -1)
    # Optionally, normalize weights to sum up to 1
    weights = weights / np.sum(weights)
    # Calculate the weighted sum for each column
    weighted_sum = np.sum(data.T * weights, axis=1)
    # Calculate the weighted average for each column
    weighted_average = weighted_sum
    return weighted_average


In [21]:
window = []
truth = []
prediction = []
for i, row in company_df.iterrows():
    if len(window) < 4:
        window.append(row['stock price'])
        continue
    window = window[1:]
    sentiment_lst = []
    for sentiment in eval(row["sentiment result"]):
        if len(sentiment) != 0:
            sentiment = sentiment[0]
            temp = [sentiment[0]["score"],sentiment[1]["score"],sentiment[2]["score"]]
            if sentiment[0]["label"]!="positive":
                print("$$error warning")
            sentiment_lst.append(temp) 

    sentiment_array = average(sentiment_lst)
    sig = np.average(np.abs(np.diff(np.array(window))))
    value = np.array([row["prediction"]-sig,row["prediction"],row["prediction"]+sig])
    p = np.array([0.25,0.5,0.25])
    print(p,sentiment_array)
    p = p*sentiment_array
    print("p",p)
    p = p/p.sum(axis=0)
    index = np.argmax(p)
    print("p",p)
    print("index",index)
    revised_prediction = value[index]
    prediction.append(revised_prediction)
    truth.append(row['stock price'])
    window.append(row['stock price'])


[0.25 0.5  0.25] [0.13646535 0.11308562 0.75044902]
p [0.03411634 0.05654281 0.18761226]
p [0.12260094 0.20319303 0.67420603]
index 2
[0.25 0.5  0.25] [0.43249166 0.19995944 0.36754897]
p [0.10812292 0.09997972 0.09188724]
p [0.36042188 0.33327698 0.30630114]
index 0
[0.25 0.5  0.25] [0.14036961 0.09368329 0.7659471 ]
p [0.0350924  0.04684165 0.19148678]
p [0.12834575 0.17131704 0.7003372 ]
index 2
[0.25 0.5  0.25] [0, 0, 0]
p [0. 0. 0.]
p [nan nan nan]
index 0
[0.25 0.5  0.25] [0.48096403 0.19352409 0.3255119 ]
p [0.12024101 0.09676205 0.08137798]
p [0.40297806 0.32429021 0.27273173]
index 0
[0.25 0.5  0.25] [0.26930591 0.20492338 0.52577072]
p [0.06732648 0.10246169 0.13144268]
p [0.2235046  0.34014341 0.43635199]
index 2
[0.25 0.5  0.25] [0.48509768 0.231869   0.28303322]
p [0.12127442 0.1159345  0.07075831]
p [0.39379002 0.37645077 0.22975921]
index 0
[0.25 0.5  0.25] [0.21631964 0.13286637 0.650814  ]
p [0.05407991 0.06643318 0.1627035 ]
p [0.19094894 0.23456671 0.57448435]
index 

/var/folders/9r/3zzn3c153yj5966w7bhc83v80000gn/T/ipykernel_43917/4120509677.py:25: RuntimeWarning: invalid value encountered in divide
  p = p/p.sum(axis=0)


In [22]:
LR_accuracy = {'Amazon': [(0.5221987315010571, 0.27138410217779574)],
 'Meta': [(0.46511627906976744, 2.9811482053333807)],
 'Google': [(0.5031712473572939, 0.2712143662161038)],
 'Microsoft': [(0.5052854122621564, 0.6500360714208605)],
 'Apple': [(0.4608879492600423, 0.20987337371763637)],
 'IBM': [(0.49471458773784355, 4.022022075505506)],
 'NVIDIA': [(0.46511627906976744, 0.00024436089268023816)],
 'Tesla': [(0.4693446088794926, 0.16787077068440348)],
 'Salesforce': [(0.5243128964059197, 2.003914772152267)],
 'AMD': [(0.4630021141649049, 0.014939979324329859)]}

In [23]:
# accuracy(amazon_truth,amazon_prediction)

print("accuracy for HMM", accuracy(np.array(truth) , np.array(prediction)))
print("accuracy for LR", LR_accuracy[company_name])

accuracy for HMM (0.4968017057569296, 1.3218899144395304)
accuracy for LR [(0.5031712473572939, 0.2712143662161038)]


predit for next day
sig = each time change difference = average change difference for that model
3 predicction =  predicction + sig , predicction, predicction - sig 
3 obs_confidence = 
final prediction = argmax(predicction * obs_confidence)

In [24]:

P =  [0.25, 0.5, 0.25]
O = np.array([ 0.4,0.2,0.4])
P = np.dot(P,O)
P = P/P.sum(axis=0)
